In [1]:
import numpy as np
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt
import random
import itertools
import time

# Load instance

In [2]:
instance_number = 1  # 0 - "kroA100.tsp"    1 - "kroB100.tsp"
instances_names = ["kroA100.tsp","kroB100.tsp"]
data = np.genfromtxt(f'instances/{instances_names[instance_number]}', skip_header=6, skip_footer=1, dtype='int64')[:,1:]

# Calculating distance matrix

In [3]:
def calc_distance_matrix(data):
    dist_matrix = distance_matrix(data, data)
    dist_matrix = np.around(dist_matrix, decimals=0)
    dist_matrix = dist_matrix.astype(int)
    return dist_matrix

In [4]:
distance_matrix = calc_distance_matrix(data)
distance_matrix

array([[   0, 2607,  549, ...,  229,  618, 1249],
       [2607,    0, 3154, ..., 2621, 3075, 2661],
       [ 549, 3154,    0, ...,  571,  403, 1499],
       ...,
       [ 229, 2621,  571, ...,    0,  480, 1475],
       [ 618, 3075,  403, ...,  480,    0, 1796],
       [1249, 2661, 1499, ..., 1475, 1796,    0]])

# GREEDY

In [5]:
def random_path(length, dataset_size):
    points = np.arange(dataset_size)
    np.random.shuffle(points)
    path, outside = points[:length], points[length:]
    return path, outside

In [6]:
def path_length(path):
	length = 0
	for i in range(len(path)-1):
		length += distance_matrix[path[i], path[i+1]]

	length += distance_matrix[path[-1], path[0]]
	return length

In [7]:
def delta_change_outside(x, out, index, distance_matrix):
    i = index[0]
    j = index[1]
    
    old = distance_matrix[x[(i-1)%50], x[i]] + distance_matrix[x[i], x[(i+1)%50]]
    new = distance_matrix[x[(i-1)%50], out[j]] + distance_matrix[out[j], x[(i+1)%50]]
       
    return old - new

In [8]:
def delta_change_inside_vertices(x, index, distance_matrix):
    i = index[0]
    j = index[1]
    if j-i == 1:
        old = distance_matrix[x[(i-1)%50], x[i]] + distance_matrix[x[i], x[j]] + distance_matrix[x[j], x[(j+1)%50]]
        new = distance_matrix[x[(i-1)%50], x[j]] + distance_matrix[x[j], x[i]] + distance_matrix[x[i], x[(j+1)%50]]
    elif j-i == 49:
        old = distance_matrix[x[(j-1)%50], x[j]] + distance_matrix[x[j], x[i]] + distance_matrix[x[i], x[(i+1)%50]]
        new = distance_matrix[x[(j-1)%50], x[i]] + distance_matrix[x[i], x[j]] + distance_matrix[x[j], x[(i+1)%50]]
    else:
        old = distance_matrix[x[(i-1)%50], x[i]] + distance_matrix[x[i], x[(i+1)%50]] + distance_matrix[x[(j-1)%50], x[j]] + distance_matrix[x[j], x[(j+1)%50]]
        new = distance_matrix[x[(i-1)%50], x[j]] + distance_matrix[x[j], x[(i+1)%50]] + distance_matrix[x[(j-1)%50], x[i]] + distance_matrix[x[i], x[(j+1)%50]]
    
    return old - new

In [9]:
def delta_change_inside_edges(x, index, distance_matrix):
    i = index[0]
    j = index[1]
    
    old = distance_matrix[x[(i-1)%50], x[i]] + distance_matrix[x[j], x[(j+1)%50]]
    new = distance_matrix[x[(i-1)%50], x[j]] + distance_matrix[x[i], x[(j+1)%50]]
    
    return old - new

In [10]:
def swap_vertices_inside(x):
    idx = []
    for i in range(len(x)):
        for j in range(i+1, len(x)):
            idx.append([i, j])
        
    idx = np.array(idx)
    return idx

In [11]:
def swap_vertices_outside(x):
    idx = []
    for i in range(len(x)):
        for j in range(len(x)):
            idx.append([i, j])
        
    idx = np.array(idx)
    return idx

In [12]:
def swap_edges(x):
    combinations = itertools.combinations(np.arange(len(x)), 2)
    return np.array([[v1,v2] for v1, v2 in combinations if 1 < v2 - v1 < len(x) - 1 ], 'int')

In [13]:
def greedy_ver(start, out):
    x = start
    N_in = swap_vertices_inside(x)
    N_out = swap_vertices_outside(x)
    np.random.shuffle(N_in)
    np.random.shuffle(N_out)
    n_i, n_o = 0, 0
    while True:
        # jeśli nie ma już żadnych ruchów
        if n_i >= len(N_in) and n_o >= len(N_out):
            break
        
        # ruch zmieniający zbiór wierzchołków 
        elif (np.random.random() < 0.5 and n_o < len(N_out)) or n_i >= len(N_in):
            idx = N_out[n_o]
            diff = delta_change_outside(x, out, idx, distance_matrix)
            if diff > 0:
                ver1 = x[idx[0]]
                ver2 = out[idx[1]]
                x[idx[0]] = ver2
                out[idx[1]] = ver1
                N_in = swap_vertices_inside(x)
                N_out = swap_vertices_outside(x)
                np.random.shuffle(N_in)
                np.random.shuffle(N_out)
                n_i, n_o = 0, 0
            else:
                n_o += 1

            
        #ruch wewnątrz trasowy - zamiana wierzchołków
        else:
            idx = N_in[n_i]
            diff =delta_change_inside_vertices(x, idx, distance_matrix)
            if diff > 0:
                x[idx[0]], x[idx[1]] = x[idx[1]], x[idx[0]]
                N_in = swap_vertices_inside(x)
                N_out = swap_vertices_outside(x)
                np.random.shuffle(N_in)
                np.random.shuffle(N_out)
                n_i, n_o = 0, 0
            else:
                n_i += 1
            
    return x

In [14]:
MIN = 100000
MAX = 0
SUM = 0
T_MIN = 100000
T_MAX = 0
T_SUM = 0

for i in range(100):
    x, out = random_path(50, 100)
    start = time.time()
    x = greedy_ver(x, out)
    t = time.time() - start
    len_x = path_length(x)
    
    if len_x > MAX:
        MAX = len_x
    if len_x < MIN:
        MIN = len_x
    SUM += len_x
    
    if t > T_MAX:
        T_MAX = t
    if t < T_MIN:
        T_MIN = t
    T_SUM += t

In [15]:
print(MIN)
print(MAX)
print(SUM/100)

12827
20139
16565.67


In [16]:
print(T_MIN)
print(T_MAX)
print(T_SUM/100)

1.6420001983642578
3.1559982299804688
2.1147722697257993


In [17]:
def greedy_edge(start, out):
    x = start
    N_in = swap_edges(x)
    N_out = swap_vertices_outside(x)
    np.random.shuffle(N_in)
    np.random.shuffle(N_out)
    n_i, n_o = 0, 0
    while True:
        # jeśli nie ma już żadnych ruchów
        if n_i >= len(N_in) and n_o >= len(N_out):
            break
        
        # ruch zmieniający zbiór wierzchołków 
        elif (np.random.random() < 0.5 and n_o < len(N_out)) or n_i >= len(N_in):
            idx = N_out[n_o]
            diff = delta_change_outside(x, out, idx, distance_matrix)
            if diff > 0:
                ver1 = x[idx[0]]
                ver2 = out[idx[1]]
                x[idx[0]] = ver2
                out[idx[1]] = ver1                
                N_in = swap_edges(x)
                N_out = swap_vertices_outside(x)
                np.random.shuffle(N_in)
                np.random.shuffle(N_out)
                n_i, n_o = 0, 0
            else:
                n_o += 1

            
        #ruch wewnątrz trasowy zamiana krawędzi
        else:
            idx = N_in[n_i]
            diff = delta_change_inside_edges(x, idx, distance_matrix)
            if diff > 0:
                a = x[idx[0]:idx[1]+1]
                a = np.flip(a)
                x[idx[0]:idx[1]+1] = a
                N_in = swap_edges(x)
                N_out = swap_vertices_outside(x)
                np.random.shuffle(N_in)
                np.random.shuffle(N_out)
                n_i, n_o = 0, 0
            else:
                n_i += 1
            
    return x

In [22]:
MIN = 100000
MAX = 0
SUM = 0
T_MIN = 100000
T_MAX = 0
T_SUM = 0

for i in range(100):
    x, out = random_path(50, 100)
    start = time.time()
    x = greedy_edge(x, out)
    t = time.time() - start
    len_x = path_length(x)
    
    if len_x > MAX:
        MAX = len_x
    if len_x < MIN:
        MIN = len_x
    SUM += len_x
    
    if t > T_MAX:
        T_MAX = t
    if t < T_MIN:
        T_MIN = t
    T_SUM += t

In [23]:
print(MIN)
print(MAX)
print(SUM/100)

11289
14665
12507.7


In [19]:
print(MIN)
print(MAX)
print(SUM/100)

11410
13719
12501.31


In [24]:
print(T_MIN)
print(T_MAX)
print(T_SUM/100)

1.499999761581421
2.462981939315796
1.9871409797668458
